<a href="https://colab.research.google.com/github/clzndyal1/git_test/blob/master/18_1_Deep_Learning_Naver_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Embedding, LSTM, Activation, Conv1D, MaxPooling1D
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import mnist, reuters, imdb
from tensorflow.keras.utils import to_categorical  #one-hot encoding과 같은 결과 반환
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import sequence

import numpy as np, array
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys

In [75]:
import urllib.request

In [76]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt",
                           filename="./data/ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt",
                           filename="./data/ratings_test.txt")

('./data/ratings_test.txt', <http.client.HTTPMessage at 0x7fde096b8ee0>)

In [77]:
train_data = pd.read_table('./data/ratings_train.txt')
test_data = pd.read_table('./data/ratings_test.txt')

In [78]:
# 데이터 전처리
    # 1. 비어 있는 리뷰 처리
    # 2. 한글외의 문자는 Nan 처리
    # 3. 같은 리뷰 제거

# 1. 비어 있는 리뷰 처리
train_data = train_data.dropna(how='any')  # null 존재하는 행 제거
# 2. 한글을 제외한 문자를 '' 로 변경
train_data['document'] = train_data['document'].str.replace(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 " "]',
                                                          '', regex=True)
# '' 를 np.nan  으로 변경 -> nan 을 제거
train_data['document'].replace('', np.nan, inplace=True)
train_data.dropna(inplace=True)
# 3. 같은 리뷰 제거
train_data.drop_duplicates(subset=['document'], inplace=True)

In [79]:
#  학습 데이터와 테스트 데이터로 정형화
    # 4. x 와 y로 분리
    # 5. 단어로 토큰
    # 6. 패딩

# 4. x 와 y로 분리
# Training data
x_train = train_data['document']
y_train = train_data['label']
# Testing data
x_test = test_data['document']
y_test = test_data['label']

In [80]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(143691,) (143691,)
(50000,) (50000,)


In [81]:
# Tokenization and padding
token_train = Tokenizer()
token_train.fit_on_texts(x_train)
train_sequences = token_train.texts_to_sequences(x_train)
test_sequences = token_train.texts_to_sequences(x_test)

AttributeError: ignored

In [ ]:
# sequence , token,-> token.rowd_index +1

In [ ]:
#모델생성
model = Sequential()   #모델 선택
model.add(Embedding(143691, 1000))   #단어 임베딩
model.add(Conv1D(200, 20))
model.add(MaxPooling1D(pool_size=4))
model.add(Dropout(0.5))        #  과적합 방지를 위한  dropout
model.add(LSTM(55))   #노드 수
model.add(Activation('relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:

model.compile(loss ='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


early_stopping = EarlyStopping(monitor='val_loss', patience=5)
hist = model.fit(x_train, y_train, epochs=200, batch_size=40, validation_split=0.25, callbacks=[early_stopping])